In [5]:
import pandas as pd
import numpy as np
from Classes import Sortino

In [3]:
prices_df = pd.read_excel("daily_dbs/daily_dbs_sp50&nasdaq.xlsx")
market_features_df = pd.read_excel("daily_dbs/daily_market_features.xlsx")
rfr_df = pd.read_csv('daily_dbs/risk_free_rate.csv')

primero vamos a llenar los NaNs que tiene el archivo de risk free rate

In [4]:
rfr_df.columns = [col.strip() for col in rfr_df.columns]
rfr_df.rename(columns={"observation_date": "Date", "DGS10": "rfr"}, inplace=True)
rfr_df['Date'] = pd.to_datetime(rfr_df['Date'])

rfr_df = rfr_df.sort_values("Date")
rfr_df['rfr'] = rfr_df['rfr'].fillna(method='ffill')

rfr_df.to_csv('daily_dbs/risk_free_rate.csv', index=False)

/var/folders/zc/nh109mkj6pld9slq4vyqbcnm0000gn/T/ipykernel_4432/121179686.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rfr_df['rfr'] = rfr_df['rfr'].fillna(method='ffill')


Vamos a generar el dataset de portafolios con pesos aleatorios y vamos a calcular su rendimiento

In [6]:
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
prices_df.set_index('Date', inplace=True)
returns_df = np.log(prices_df/prices_df.shift(1)).dropna()
returns_df.to_csv('daily_dbs/returns_df.csv')

selección aleatoria de activos

In [7]:
np.random.seed(42)
n = 20
tickers = returns_df.columns.tolist()
selected_assets = np.random.choice(tickers, n, replace=False).tolist()
selected_assets

['LIN',
 'TECH',
 'SNA',
 'IP',
 'NRG',
 'TDG',
 'YUM',
 'DTE',
 'EXPE',
 'ROST',
 'V',
 'MS',
 'AEP',
 'J',
 'VMC',
 'RJF',
 'BDX',
 'ACN',
 'LMT',
 'REG']

llamamos a la clase de sortino que hace toda la chamba :V

In [9]:
rfr_path = 'daily_dbs/risk_free_rate.csv'
sortino = Sortino(returns_df, rfr_path, selected_assets)

sortino.generate_multiple_weights(num_combinations=1000)
sortino.calculate_portfolio_returns()
sortino.calculate_sortino_ratio()

sortino_dataset = sortino.create_portfolio_dataset()
sortino_dataset.head()

Generadas 1000 combinaciones de pesos para cada fecha.


KeyboardInterrupt: 